In [1]:
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import cleanlab
import pickle

In [2]:
df=pd.read_csv(r"C:\Users\tareq\Desktop\p2\Data\All_Users_Labeled.csv")

In [3]:
df.drop(['Unnamed: 0','Username'],axis=1,inplace=True)

In [4]:
y = df['c'].values

In [5]:
X_num = df.iloc[:, 1:5].values
ss=MinMaxScaler()
X_num=ss.fit_transform(X=X_num)
vectorizer = CountVectorizer()
X_text = vectorizer.fit_transform(df['Full_Text'])
df1=pd.DataFrame(X_text.toarray(),columns=vectorizer.get_feature_names_out())
df1["total_words"]= [x[0] for x in X_num]
df1["total_char"]= [x[1] for x in X_num]
df1["avg_chr_per_word"]= [x[2] for x in X_num]
df1["total_emojis"]= [x[3] for x in X_num]

In [11]:
X_train, X_test, y_train, y_test= train_test_split(df1, y, test_size=0.2, random_state=42)

In [ ]:
ada_parameters = {'n_estimators': [25, 50, 100,200,250], 'learning_rate': [0.5, 1, 1.5,1.75],'random_state':[42]}
ada = AdaBoostClassifier()
ada_grid = GridSearchCV(ada, ada_parameters, cv=5)
ada_grid.fit(X_train, y_train)

In [ ]:
cl = cleanlab.classification.CleanLearning(ada_grid.best_estimator_,seed=42)
cl.fit(X_train, y_train)

In [10]:
y_pred=cl.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       271
           1       0.77      0.81      0.79        59

    accuracy                           0.92       330
   macro avg       0.87      0.88      0.87       330
weighted avg       0.93      0.92      0.92       330



In [ ]:
pickle.dump(cl,open('final_ml_models/C.pkl', 'wb'))